Following [here](https://statsthinking21.github.io/statsthinking21-python/08-HypothesisTesting.html)

[Other reference](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html)

In [1]:
# Import packages

import numpy as np
import pandas as pd
import scipy.stats as sds


In [2]:
# Import talk data

# Load binary gendered talks 
talks_male = pd.read_csv('talks_male.csv', index_col='Talk_ID')
talks_female = pd.read_csv('talks_female.csv', index_col='Talk_ID')

# No one gender ==> NOG
talks_nog = pd.read_csv('talks_nog.csv', index_col='Talk_ID')

# Concatenate the talk data
all_talks = pd.concat([talks_male,talks_female,talks_nog])

In [23]:
# Import speaker data

speakers = pd.read_csv('speakers_with_gender.csv', index_col='speaker')

In [4]:
# Create list of events

events = list(all_talks['event'].unique())

In [22]:
year = '2007'
event_name = 'TED' + str(year)
event_name

'TED2007'

In [5]:
event_year = all_talks[all_talks['event'] == 'TED2007']


In [27]:
speaker_list = extract_speakers_by_year(2007)

# How to check if the list is unique?? Extend somehow? 
# Done see above

In [43]:
gender_year = speakers.loc[speaker_list]
gender_counts = gender_year.groupby(["Gender_handcheck"])["Gender_handcheck"].count()

gender_counts["female"]

15

In [27]:
# Create the sampling distributions

def generate_one_sample_prop(num_sample = 100, null_prop = 0.5):
    pass

def generate_sampling_dist(num_runs = 5000):
    pass



In [25]:
def extract_talks_by_year(year):
    
    #Check if year is in list: 
    event_name = 'TED' + str(year)
    assert (event_name in events), "There are no talks in our database from the requested year"
    
    event_year = all_talks[all_talks['event'] == event_name]
    
    return event_year

def extract_speakers_by_year(year):
    
    talks_year = extract_talks_by_year(year)
    event_speakers = talks_year[['speaker_1', 'speaker_2', 'speaker_3', 'speaker_4']]
    
    speaker_list = []
    
    for role in ['speaker_1', 'speaker_2', 'speaker_3', 'speaker_4']:
        event_speakers = event_year[role].dropna()
        event_speakers = event_speakers.unique()
        speaker_list.extend(list(event_speakers))
        
    speaker_pd = pd.Series(speaker_list)
    speaker_pd = speaker_pd.unique()
    
    speaker_year = list(speaker_array)
    ## Ref: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
    
    return speaker_year
    
    
def prop_women_by_year(year):
    
    speaker_year = extract_speakers_by_year(year)
    gender_year = speakers.loc[speaker_year]
    gender_counts = gender_year.groupby(["Gender_handcheck"])["Gender_handcheck"].count()
    
    
    ## Ref: https://sparkbyexamples.com/pandas/pandas-groupby-count-examples/